In [1]:
import geopandas as gpd
import pyarrow
import pandas as pd
import os

In [2]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [4]:
n = len(target_2631)


4724838

In [6]:
target_2631.loc[0]

utc_timestamp                                           1572117649
caid             0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e3...
geo_hash                                                 9qbdwqx8v
latitude                                                   38.4441
longitude                                                 -122.762
utc_datetime                             2019-10-26 19:20:49+00:00
la_datetime                              2019-10-26 12:20:49-07:00
hour                                                            12
lat_epsg_3857                                         -1.36658e+07
lon_epsg_3857                                          4.64235e+06
Name: 0, dtype: object

In [31]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
start_time = target_2631.loc[start_index]['la_datetime']
end_time = target_2631.loc[start_index]['la_datetime']
data = {'index': [-1], 'caid': [start_id], 'start_time' : [start_time], 'end_time' : [end_time], 'count' : [0], 'is_process' : [False], 'is_success' : [False], 'res' : ['LINESTRING()'] }  
df_new = pd.DataFrame(columns=['index', 'caid', 'start_time', 'end_time', 'count', 'is_process', 'is_success', 'res'])
# Create DataFrame  
df_new = pd.DataFrame(data)
print("start index: %d, caid: %s" %(0, start_id))
end = n
#end = 10000
count = 0
is_first = True
res = 'LINESTRING('
for index in range(0, end):
    data_id = target_2631.loc[index]['caid']
    single_data = target_2631.loc[index]
  # is_data_valid = (single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)
  
    if data_id != start_id:
    # print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
        res+=')'
        df_new.loc[count] = {'index': count, 'caid': target_2631.loc[index - 1]['caid'], 'start_time' : start_time, 'end_time' : target_2631.loc[index - 1]['la_datetime'], 'count' : (index - start_index), 'is_process' : False, 'is_success' : False, 'res' : res }
        is_first = True
        res = 'LINESTRING('
        count+=1
        start_index = index
        start_id = data_id
        start_time = target_2631.loc[index]['la_datetime']
    lat = float(single_data['latitude'])
    lon = float(single_data['longitude'])
    if is_first:
        is_first = False
        res+= (str(lon) + ' ' + str(lat))
    else:
        res+= (',' + str(lon) + ' ' + str(lat))

start index: 0, caid: 0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e35c30d5d68a0008feb6


In [32]:
df_new.to_csv('data/GPS.nosync/Sonoma-2631-new-points/out2.csv')

In [33]:
df = pd.read_csv('data/GPS.nosync/Sonoma-2631-new-points/out2.csv')

In [48]:
time = df['start_time'][4]
print(time)
pd.Timestamp(time)

2019-10-25 17:03:02-07:00


Timestamp('2019-10-25 17:03:02-0700', tz='pytz.FixedOffset(-420)')